# Paper C - Experiment E3: Learning Demo (本番)

## 較正済み設定
- **center_scale = 2.5**
- **noise_std = 1.0**
- SNR = 2.5
- Clean training test accuracy: 79.9% (in 70-85% target)

## 実験設計
- Noise levels: 0%, 30%, 50% (3水準)
- ρ: -0.7, -0.4, -0.2, 0.0, +0.4, +0.7, +1.0 (7水準)
- λ: 0.00, 0.25, 0.50, 0.75, 1.00 (5水準)
- Seeds: 10
- **Total: 1,050 runs**

## 出力
- Fig C5: Accuracy surface (ρ × λ)
- Fig C6: Effect of ρ at different λ
- Fig C7: cos(g_val, g_ref) verification
- Fig C8: Confounding visualization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
SAVE_DIR = '/content/drive/MyDrive/paper-C-results/E3_production'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

print(f'Started: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

In [ ]:
# ========================================
# 較正済み設定
# ========================================

# データ生成パラメータ（較正済み）
CENTER_SCALE = 2.5  # ★ 較正済み
NOISE_STD_DATA = 1.0  # ★ 較正済み
N_TRAIN = 20000
N_TEST = 10000
N_CLASSES = 10
DIM_PER_VIEW = 8
CENTER_SEED = 0

# 実験グリッド
NOISE_RATES = [0.0, 0.3, 0.5]  # ラベルノイズ率
RHO_VALUES = [-0.7, -0.4, -0.2, 0.0, 0.4, 0.7, 1.0]
LAMBDA_VALUES = [0.0, 0.25, 0.5, 0.75, 1.0]
SEEDS = list(range(10))

# 学習パラメータ
EPOCHS = 100
LR = 0.1

TOTAL_RUNS = len(NOISE_RATES) * len(RHO_VALUES) * len(LAMBDA_VALUES) * len(SEEDS)

print('=' * 60)
print('E3 Production: Learning Demo')
print('=' * 60)
print(f'\n■ 較正済み設定:')
print(f'  center_scale = {CENTER_SCALE}')
print(f'  noise_std = {NOISE_STD_DATA}')
print(f'  Expected clean accuracy: ~80%')
print(f'\n■ 実験グリッド:')
print(f'  Noise rates: {NOISE_RATES}')
print(f'  ρ values: {RHO_VALUES}')
print(f'  λ values: {LAMBDA_VALUES}')
print(f'  Seeds: {len(SEEDS)}')
print(f'\n■ Total runs: {TOTAL_RUNS}')
print('=' * 60)

In [ ]:
# データ生成関数

def generate_orthogonal_centers(n_classes, dim_per_view, center_scale, seed=0):
    rng = np.random.RandomState(seed)
    total_dim = dim_per_view * 2
    random_matrix = rng.randn(total_dim, n_classes)
    Q, _ = np.linalg.qr(random_matrix)
    centers_joint = Q[:, :n_classes].T * center_scale
    centers_A = centers_joint[:, :dim_per_view]
    centers_B = centers_joint[:, dim_per_view:]
    return centers_A.astype(np.float32), centers_B.astype(np.float32)

def generate_two_view_data(n_samples, centers_A, centers_B, noise_std, sample_seed):
    n_classes = centers_A.shape[0]
    dim_per_view = centers_A.shape[1]
    rng = np.random.RandomState(sample_seed)
    labels = rng.randint(0, n_classes, n_samples)
    view_A = np.array([centers_A[l] + rng.randn(dim_per_view) * noise_std for l in labels])
    view_B = np.array([centers_B[l] + rng.randn(dim_per_view) * noise_std for l in labels])
    X = np.concatenate([view_A, view_B], axis=1)
    return X.astype(np.float32), labels.astype(np.int64)

def inject_label_noise(labels, noise_rate, n_classes=10, seed=42):
    if noise_rate == 0:
        return labels.copy()
    rng = np.random.RandomState(seed + 1000)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    indices = rng.choice(len(labels), n_noisy, replace=False)
    for idx in indices:
        noisy[idx] = rng.choice([i for i in range(n_classes) if i != labels[idx]])
    return noisy

print('データ生成関数定義完了')

In [ ]:
# モデル定義

class TwoViewMLP(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=64, n_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_classes)
        )
    
    def forward(self, x):
        return self.net(x)

print('モデル定義完了')

In [ ]:
# 勾配操作関数

def get_gradient_vector(model):
    """モデルの全パラメータの勾配を1つのベクトルとして取得"""
    grads = []
    for p in model.parameters():
        if p.grad is not None:
            grads.append(p.grad.view(-1))
    return torch.cat(grads)

def set_gradient_vector(model, grad_vec):
    """ベクトルをモデルの各パラメータの勾配として設定"""
    idx = 0
    for p in model.parameters():
        numel = p.numel()
        p.grad = grad_vec[idx:idx+numel].view(p.shape).clone()
        idx += numel

def orthogonalize(g_ref_norm, v):
    """vからg_ref_norm方向の成分を除去し、正規化"""
    v_perp = v - torch.dot(v, g_ref_norm) * g_ref_norm
    norm = torch.norm(v_perp)
    if norm < 1e-10:
        # 万が一平行な場合はランダムベクトルで再試行
        v = torch.randn_like(v)
        v_perp = v - torch.dot(v, g_ref_norm) * g_ref_norm
        norm = torch.norm(v_perp)
    return v_perp / norm

def construct_g_value(g_ref_norm, rho, device):
    """
    ρ-design: cos(g_value, g_ref) = ρ となるg_valueを構築
    g_value = ρ * g_ref_norm + sqrt(1-ρ²) * g_perp
    """
    # g_refに直交するランダムベクトルを生成
    random_vec = torch.randn_like(g_ref_norm)
    g_perp = orthogonalize(g_ref_norm, random_vec)
    
    # ρに基づいてg_valueを構築
    if abs(rho) >= 1.0:
        g_value = rho * g_ref_norm
    else:
        sqrt_term = np.sqrt(1 - rho**2)
        g_value = rho * g_ref_norm + sqrt_term * g_perp
    
    # 正規化
    g_value = g_value / torch.norm(g_value)
    return g_value, g_perp

print('勾配操作関数定義完了')

In [ ]:
# 実験実行関数

def run_experiment(noise_rate, rho, lam, model_seed):
    """
    1つの(noise_rate, ρ, λ, seed)設定で学習を実行
    
    Returns:
        dict: 結果（accuracy, cos値など）
    """
    # クラス中心を生成（固定）
    centers_A, centers_B = generate_orthogonal_centers(
        N_CLASSES, DIM_PER_VIEW, CENTER_SCALE, seed=CENTER_SEED
    )
    
    # データ生成
    X_train, y_train_clean = generate_two_view_data(
        N_TRAIN, centers_A, centers_B, NOISE_STD_DATA, sample_seed=42
    )
    X_test, y_test = generate_two_view_data(
        N_TEST, centers_A, centers_B, NOISE_STD_DATA, sample_seed=43
    )
    
    # ラベルノイズ注入
    y_train_noisy = inject_label_noise(y_train_clean, noise_rate, N_CLASSES, seed=model_seed)
    
    # テンソル化
    X_train_t = torch.tensor(X_train, device=device)
    y_clean_t = torch.tensor(y_train_clean, device=device)
    y_noisy_t = torch.tensor(y_train_noisy, device=device)
    X_test_t = torch.tensor(X_test, device=device)
    y_test_t = torch.tensor(y_test, device=device)
    
    # モデル初期化
    set_seed(model_seed)
    model = TwoViewMLP().to(device)
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    
    # cos値の記録用
    cos_values = []
    
    # 学習ループ
    for epoch in range(EPOCHS):
        model.train()
        
        # Step 1: g_ref（clean labels）の計算
        optimizer.zero_grad()
        loss_ref = criterion(model(X_train_t), y_clean_t)
        loss_ref.backward()
        g_ref = get_gradient_vector(model).clone()
        g_ref_norm = g_ref / torch.norm(g_ref)
        g_ref_scale = torch.norm(g_ref)  # 元のスケールを保存
        
        # Step 2: g_struct（noisy labels）の計算
        optimizer.zero_grad()
        loss_struct = criterion(model(X_train_t), y_noisy_t)
        loss_struct.backward()
        g_struct = get_gradient_vector(model).clone()
        g_struct_norm = g_struct / torch.norm(g_struct)
        g_struct_scale = torch.norm(g_struct)
        
        # Step 3: g_value（ρ-design）の構築
        g_value, _ = construct_g_value(g_ref_norm, rho, device)
        
        # cos検証（記録用）
        cos_actual = torch.dot(g_value, g_ref_norm).item()
        cos_values.append(cos_actual)
        
        # Step 4: 勾配の混合 g_mix = λ * g_struct + (1-λ) * g_value
        g_mix = lam * g_struct_norm + (1 - lam) * g_value
        g_mix = g_mix / torch.norm(g_mix)  # 正規化
        
        # スケール復元（g_structのスケールを使用）
        g_mix_scaled = g_mix * g_struct_scale
        
        # Step 5: 勾配を設定して更新
        optimizer.zero_grad()
        set_gradient_vector(model, g_mix_scaled)
        optimizer.step()
    
    # 評価
    model.eval()
    with torch.no_grad():
        train_acc = (model(X_train_t).argmax(1) == y_clean_t).float().mean().item()
        test_acc = (model(X_test_t).argmax(1) == y_test_t).float().mean().item()
    
    return {
        'noise_rate': noise_rate,
        'rho': rho,
        'lambda': lam,
        'seed': model_seed,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'cos_mean': np.mean(cos_values),
        'cos_std': np.std(cos_values),
        'cos_final': cos_values[-1] if cos_values else 0
    }

print('実験実行関数定義完了')

In [ ]:
# ★ 動作確認（1設定）

print('=== 動作確認 ===')
result = run_experiment(noise_rate=0.3, rho=0.7, lam=0.5, model_seed=0)
print(f'noise=30%, ρ=0.7, λ=0.5')
print(f'  test_acc = {result["test_acc"]:.3f}')
print(f'  cos(g_val, g_ref) = {result["cos_mean"]:.4f} (target: 0.7)')

if abs(result['cos_mean'] - 0.7) < 0.05:
    print('\n✓ ρ-design検証OK')
else:
    print('\n⚠️ cos値がターゲットからずれています')

In [ ]:
# ========================================
# 本番実験の実行
# ========================================

print('\n' + '=' * 60)
print('E3 Production: Main Experiment')
print(f'Total: {TOTAL_RUNS} runs')
print('=' * 60 + '\n')

results = []
start_time = time.time()
run_count = 0

for noise_rate in NOISE_RATES:
    noise_pct = int(noise_rate * 100)
    print(f'\n=== NOISE {noise_pct}% ===')
    
    for rho in RHO_VALUES:
        for lam in LAMBDA_VALUES:
            for seed in SEEDS:
                run_count += 1
                result = run_experiment(noise_rate, rho, lam, seed)
                results.append(result)
                
                # 進捗表示（50回ごと）
                if run_count % 50 == 0:
                    elapsed = time.time() - start_time
                    eta = elapsed / run_count * (TOTAL_RUNS - run_count)
                    print(f'  [{run_count:4d}/{TOTAL_RUNS}] '
                          f'noise={noise_pct}% ρ={rho:+.1f} λ={lam:.2f} | '
                          f'acc={result["test_acc"]:.3f} | '
                          f'ETA: {eta/60:.1f}min')

total_time = time.time() - start_time
print(f'\n' + '=' * 60)
print(f'完了！ Total time: {total_time/60:.1f} min')
print('=' * 60)

In [ ]:
# 結果をDataFrameに変換

df = pd.DataFrame(results)

# 集計（seed平均）
df_agg = df.groupby(['noise_rate', 'rho', 'lambda']).agg({
    'train_acc': ['mean', 'std'],
    'test_acc': ['mean', 'std'],
    'cos_mean': ['mean', 'std']
}).reset_index()
df_agg.columns = ['noise_rate', 'rho', 'lambda', 
                  'train_acc_mean', 'train_acc_std',
                  'test_acc_mean', 'test_acc_std',
                  'cos_mean', 'cos_std']

print('結果集計完了')
print(f'総レコード数: {len(df)}')
print(f'集計後レコード数: {len(df_agg)}')

In [ ]:
# Fig C5: Accuracy Surface (ρ × λ)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, noise_rate in enumerate(NOISE_RATES):
    ax = axes[idx]
    noise_pct = int(noise_rate * 100)
    
    df_noise = df_agg[df_agg['noise_rate'] == noise_rate]
    pivot = df_noise.pivot(index='rho', columns='lambda', values='test_acc_mean')
    
    im = ax.imshow(pivot.values, cmap='RdYlGn', aspect='auto', 
                   vmin=0.1, vmax=1.0, origin='lower')
    
    ax.set_xticks(range(len(LAMBDA_VALUES)))
    ax.set_xticklabels([f'{l:.2f}' for l in LAMBDA_VALUES])
    ax.set_yticks(range(len(RHO_VALUES)))
    ax.set_yticklabels([f'{r:+.1f}' for r in RHO_VALUES])
    ax.set_xlabel('λ', fontsize=12)
    ax.set_ylabel('ρ', fontsize=12)
    ax.set_title(f'Noise = {noise_pct}%', fontsize=14, fontweight='bold')
    
    # 値を表示
    for i in range(len(RHO_VALUES)):
        for j in range(len(LAMBDA_VALUES)):
            val = pivot.values[i, j]
            color = 'white' if val < 0.5 else 'black'
            ax.text(j, i, f'{val:.2f}', ha='center', va='center', 
                   fontsize=8, color=color)

plt.colorbar(im, ax=axes, label='Test Accuracy', shrink=0.8)
fig.suptitle('Fig C5: Accuracy Surface', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/fig_C5_accuracy_surface.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Fig C5 保存完了')

In [ ]:
# Fig C6: Effect of ρ at Different λ

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
colors = plt.cm.viridis(np.linspace(0, 1, len(LAMBDA_VALUES)))

for idx, noise_rate in enumerate(NOISE_RATES):
    ax = axes[idx]
    noise_pct = int(noise_rate * 100)
    
    df_noise = df_agg[df_agg['noise_rate'] == noise_rate]
    
    for lam_idx, lam in enumerate(LAMBDA_VALUES):
        df_lam = df_noise[df_noise['lambda'] == lam]
        ax.errorbar(df_lam['rho'], df_lam['test_acc_mean'], 
                   yerr=df_lam['test_acc_std'],
                   marker='o', label=f'λ={lam:.2f}', 
                   color=colors[lam_idx], capsize=3)
    
    ax.axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='Chance')
    ax.set_xlabel('ρ', fontsize=12)
    ax.set_ylabel('Test Accuracy', fontsize=12)
    ax.set_title(f'Noise = {noise_pct}%', fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=8)
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)

fig.suptitle('Fig C6: Effect of ρ at Different λ', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/fig_C6_rho_effect.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Fig C6 保存完了')

In [ ]:
# Fig C7: cos(g_val, g_ref) Verification

fig, ax = plt.subplots(figsize=(8, 6))

# ρごとに集計
df_cos = df.groupby('rho')['cos_mean'].agg(['mean', 'std']).reset_index()

ax.errorbar(df_cos['rho'], df_cos['mean'], yerr=df_cos['std'],
           marker='o', markersize=10, capsize=5, linewidth=2, label='Measured')

# 理想線
ax.plot(RHO_VALUES, RHO_VALUES, 'r--', linewidth=2, label='Ideal (cos = ρ)')

ax.set_xlabel('Target ρ', fontsize=14)
ax.set_ylabel('Measured cos(g_val, g_ref)', fontsize=14)
ax.set_title('Fig C7: ρ-Design Verification', fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xlim(-1, 1.1)
ax.set_ylim(-1, 1.1)

# 誤差統計を表示
cos_deviation = np.abs(df_cos['mean'].values - df_cos['rho'].values)
ax.text(0.05, 0.95, f'Max deviation: {cos_deviation.max():.6f}',
       transform=ax.transAxes, fontsize=10, verticalalignment='top',
       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/fig_C7_cos_verification.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Fig C7 保存完了')
print(f'  cos検証: max deviation = {cos_deviation.max():.6f}')

In [ ]:
# Fig C8: Confounding Visualization
# A_naive = cos(g_mix, g_ref) がλで変動することを示す

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, noise_rate in enumerate(NOISE_RATES):
    ax = axes[idx]
    noise_pct = int(noise_rate * 100)
    
    df_noise = df_agg[df_agg['noise_rate'] == noise_rate]
    
    # ρを固定してλによるcos_mean（≈A_naive）の変動を見る
    for rho in [0.0, 0.4, 0.7, 1.0]:
        df_rho = df_noise[df_noise['rho'] == rho]
        if len(df_rho) > 0:
            ax.plot(df_rho['lambda'], df_rho['test_acc_mean'], 
                   marker='o', label=f'ρ={rho:+.1f}')
    
    ax.set_xlabel('λ', fontsize=12)
    ax.set_ylabel('Test Accuracy', fontsize=12)
    ax.set_title(f'Noise = {noise_pct}%', fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1)

fig.suptitle('Fig C8: Effect of λ at Fixed ρ (Confounding Check)', 
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/fig_C8_confounding.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Fig C8 保存完了')

In [ ]:
# 結果の保存

# 生データ
df.to_csv(f'{SAVE_DIR}/E3_results_raw.csv', index=False)

# 集計データ
df_agg.to_csv(f'{SAVE_DIR}/E3_results_aggregated.csv', index=False)

# サマリーJSON
summary = {
    'experiment': 'E3_production',
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'calibrated_settings': {
        'center_scale': CENTER_SCALE,
        'noise_std': NOISE_STD_DATA,
        'snr': CENTER_SCALE / NOISE_STD_DATA
    },
    'grid': {
        'noise_rates': NOISE_RATES,
        'rho_values': RHO_VALUES,
        'lambda_values': LAMBDA_VALUES,
        'n_seeds': len(SEEDS)
    },
    'total_runs': TOTAL_RUNS,
    'cos_verification': {
        'max_deviation': float(cos_deviation.max()),
        'mean_deviation': float(cos_deviation.mean())
    },
    'accuracy_summary': {
        'overall_mean': float(df['test_acc'].mean()),
        'overall_std': float(df['test_acc'].std()),
        'by_noise': {f'{int(nr*100)}%': float(df[df['noise_rate']==nr]['test_acc'].mean()) 
                    for nr in NOISE_RATES}
    }
}

with open(f'{SAVE_DIR}/E3_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print('\n=== 保存完了 ===')
print(f'  {SAVE_DIR}/E3_results_raw.csv')
print(f'  {SAVE_DIR}/E3_results_aggregated.csv')
print(f'  {SAVE_DIR}/E3_summary.json')
print(f'  {SAVE_DIR}/fig_C5_accuracy_surface.png')
print(f'  {SAVE_DIR}/fig_C6_rho_effect.png')
print(f'  {SAVE_DIR}/fig_C7_cos_verification.png')
print(f'  {SAVE_DIR}/fig_C8_confounding.png')

In [ ]:
# 最終サマリー

print('\n' + '=' * 70)
print('E3 Production 完了')
print('=' * 70)

print('\n■ 較正済み設定:')
print(f'  center_scale = {CENTER_SCALE}')
print(f'  noise_std = {NOISE_STD_DATA}')
print(f'  SNR = {CENTER_SCALE / NOISE_STD_DATA}')

print('\n■ ρ-design検証:')
print(f'  cos(g_val, g_ref) = ρ')
print(f'  Max deviation: {cos_deviation.max():.6f}')

print('\n■ 精度サマリー:')
for noise_rate in NOISE_RATES:
    noise_pct = int(noise_rate * 100)
    mean_acc = df[df['noise_rate'] == noise_rate]['test_acc'].mean()
    std_acc = df[df['noise_rate'] == noise_rate]['test_acc'].std()
    print(f'  Noise {noise_pct}%: {mean_acc:.1%} ± {std_acc:.1%}')

print('\n■ 出力ファイル:')
print('  - Fig C5: Accuracy Surface')
print('  - Fig C6: Effect of ρ at Different λ')
print('  - Fig C7: cos Verification')
print('  - Fig C8: Confounding Check')

print('\n' + '=' * 70)